In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,2,'Sprint')
session.load()

In [ ]:
drivers = session.drivers

In [ ]:
grid_pos = session.results.sort_values('GridPosition')
grid_order = grid_pos['Abbreviation'].values
starting_positions = {driver: i+1 for i, driver in enumerate(grid_order)}

In [ ]:
start_row = pd.Series(starting_positions)

In [ ]:
pos_dict = {}
for driver_number in drivers:
    temp = session.laps.pick_drivers(driver_number)
    pos_dict[temp.Driver.iloc[0]] = temp.Position.values

pos_df = pd.DataFrame(pos_dict)

In [ ]:
pos_df = pd.concat([start_row.to_frame().T, pos_df], ignore_index=True)

In [ ]:
pos_df

In [ ]:
n_interp = 20  # number of subframes between laps

# Create a new index for interpolation
new_index = np.linspace(0, len(pos_df) - 1, (len(pos_df) - 1) * n_interp + 1)

# Interpolate all drivers (columns)
pos_df_interp = pos_df.reindex(new_index).interpolate(method='linear', axis=0).reset_index(drop=True)

In [ ]:
pos_df_interp

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
drivers = pos_df.columns.tolist()

# For each driver: keep line and head marker
lines = {}
heads = {}
texts = {}

for i, driver in enumerate(drivers):
    color = plotting.get_driver_style(driver,style=['color'],session=session)['color']
    (line,) = ax.plot([], [], color=color, lw=2, label=driver)
    (head,) = ax.plot([], [], 'o', color=color, markersize=6)
    txt = ax.text(0, 0, driver, fontsize=12, fontweight='bold', color=color, va='center', ha='left')
    lines[driver] = line
    heads[driver] = head
    texts[driver] = txt

ax.set_ylim(20.5, 0.5)  # Invert y-axis: P1 at top
ax.set_xlim(0, 5)       # Initial lap window
ax.set_yticks([i for i in range(1,21)])
ax.set_xlabel("Lap")
ax.set_ylabel("Position on Track")
ax.set_title("Sprint Position Evolution")
ax.grid(visible=False)
# legend = ax.legend(loc='upper right')

def update(frame):
    x = np.linspace(0, frame / n_interp, frame + 1)
    
    for driver in drivers:
        y_data = pos_df_interp[driver].iloc[:frame + 1]
        lines[driver].set_data(x, y_data)
        heads[driver].set_data([x[-1]], [y_data.iloc[-1]])  # Head marker at last point
        texts[driver].set_position((x[-1] + 0.1, y_data.iloc[-1])) # Text on the right side of markers
 
    # Scroll window: show 5-lap window
    current_lap = frame / n_interp
    if current_lap > 2.5:
        ax.set_xlim(current_lap - 2.5, current_lap + 2.5)
    else:
        ax.set_xlim(0, 5)

    return list(lines.values()) + list(heads.values())

n_frames = len(pos_df_interp)
ani = FuncAnimation(fig, update, frames=n_frames, interval=1000/n_interp, blit=True, repeat=False)

# To display:
plt.tight_layout()
HTML(ani.to_jshtml())